In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
! ls -al

In [ ]:
project_id = 'leftover-199123'
!gcloud config set project {project_id}

In [ ]:
! gsutil cp gs://advml-bucket/ads.pickle .
! gsutil cp gs://advml-bucket/filtered_events.csv .

In [1]:
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
import keras
import scipy

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def getAdIdConverterFunction(adIds):
    # adIdsRev is a dict mapping from ad_id to ad_inx (i.e. the inx of such ad in adIds)
    adIdsRev = {adId: inx for inx, adId in enumerate(adIds)}
    # Vectorized function to convert ad_id into ad_inx
    convertToAdInx = np.vectorize(lambda adId: adIdsRev[adId])
    return (adIdsRev,convertToAdInx)

def loadAdWeightsAndIds():
    with open('ads.pickle','rb') as f:
        obj = pickle.load(f)
    return obj

sparseAdWeights,adIds = loadAdWeightsAndIds()
adIdsRev,convertToAdInx = getAdIdConverterFunction(adIds)

eventsDf = pd.read_csv('filtered_events.csv')

uniqUser = np.unique(eventsDf.uuid)

In [42]:
from keras.layers import *


def probit_activation(x):
    return tf.distributions.Normal(loc=0., scale=1.).cdf(x)

def createProbitModel():

#     userInxInput = Input(shape=(1,))
    adInxInput = Input(shape=(1,))

    adWeightLayer = Embedding(
        sparseAdWeights.shape[0],
        sparseAdWeights.shape[1],
        input_length=1,
        trainable=False,
        weights=[sparseAdWeights.toarray()]
    )(adInxInput)

#     userWeightLayer = Embedding(uniqUser.shape[0],sparseAdWeights.shape[1],input_length=1)(userInxInput)

#     dotLayer = Dot(-1)([adWeightLayer,userWeightLayer])
    flat_ = Dense(1)(adWeightLayer)

    flat_ = Flatten()(flat_)
#     flat_ = Flatten()(dotLayer)

    activationLayer = Activation(probit_activation)(flat_)

    model = keras.models.Model(inputs=(adInxInput),outputs=(activationLayer))

    model.compile(loss='mse', optimizer='adam') ## Maybe another optimizer?
    
    return model

model = createProbitModel()

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 1)                 0         
_________________________________________________________________
embedding_10 (Embedding)     (None, 1, 96)             53719968  
_________________________________________________________________
dense_8 (Dense)              (None, 1, 1)              97        
_________________________________________________________________
flatten_5 (Flatten)          (None, 1)                 0         
_________________________________________________________________
activation_7 (Activation)    (None, 1)                 0         
Total params: 53,720,065
Trainable params: 97
Non-trainable params: 53,719,968
_________________________________________________________________


In [43]:
from keras.callbacks import *

weights_filename = 'probit.h5'

model.fit(
    [eventsDf.ad_inx],
    eventsDf.clicked,
    epochs = 30,
    shuffle=True,
    batch_size=2000,
    callbacks=[
        EarlyStopping(monitor='loss', patience=2),
        ModelCheckpoint(weights_filename, monitor='loss', save_best_only=True, save_weights_only=True),
    ]
)

Epoch 1/30
2405074/2405074 [==============================] - 5s 2us/step - loss: 0.1611
Epoch 2/30
2405074/2405074 [==============================] - 4s 2us/step - loss: 0.1521
Epoch 3/30
2405074/2405074 [==============================] - 4s 2us/step - loss: 0.1520
Epoch 4/30
2405074/2405074 [==============================] - 4s 2us/step - loss: 0.1519
Epoch 5/30
2405074/2405074 [==============================] - 4s 2us/step - loss: 0.1519
Epoch 6/30
2405074/2405074 [==============================] - 4s 2us/step - loss: 0.1519
Epoch 7/30
2405074/2405074 [==============================] - 4s 2us/step - loss: 0.1519
Epoch 8/30
2405074/2405074 [==============================] - 4s 2us/step - loss: 0.1519
Epoch 9/30
2405074/2405074 [==============================] - 4s 2us/step - loss: 0.1519
Epoch 10/30
2405074/2405074 [==============================] - 5s 2us/step - loss: 0.1519
Epoch 11/30
2405074/2405074 [==============================] - 5s 2us/step - loss: 0.1519
Epoch 12/30
2405074

In [ ]:
! gsutil cp probit.h5 gs://advml-bucket/

In [ ]:
! gsutil cp gs://advml-bucket/probit.h5 .
    
model.load_weights('probit.h5')

## simulation

In [14]:



class SubModDiv():
    def __init__(self,wAds,regrModel,alpha=1.0,beta=1.0):
        self.regrModel = regrModel
        
        self.a = alpha
        self.b = beta
        self.wAds = wAds.toarray() # "a_i" in paper
        self.c = np.zeros(wAds.shape[1]) # "c" in section 4.1 of paper
        self.v = np.zeros(wAds.shape[1]) # "v" in section 4.1 of paper
        
#         with tf.device(' ')
        with tf.device('/gpu:0'):
            self.initTensorflowOp()
        
    def initTensorflowOp(self):
        ## Tensorflow code for subsetiteration
        t_prevAdInx = tf.placeholder(tf.int32,shape=(None))
        t_probs = tf.placeholder(tf.float32,shape=(self.wAds.shape[0]))
        t_wAds = tf.placeholder(tf.float32,shape=self.wAds.shape)
        t_w = tf.placeholder(tf.float32,shape=(self.wAds.shape[1]))
        
        t_prevAdSum = tf.constant(1.0)+tf.reduce_sum(tf.gather(t_wAds,t_prevAdInx),axis=0)
        
        t_prevProbSum = tf.reduce_sum(tf.gather(t_probs,t_prevAdInx),axis=0)
        
        t_newAs = tf.log(t_wAds + t_prevAdSum)
        
        t_dotProds = tf.reduce_sum(t_w * t_newAs,axis=1)
        
        t_scores = t_prevProbSum + t_probs
        
        t_prevAdMask = tf.cond(
            tf.equal( tf.shape(t_prevAdInx)[0], tf.constant(0,dtype=tf.int32) ),
            lambda: tf.zeros(self.wAds.shape[0]),
            lambda: tf.reduce_sum(tf.one_hot(t_prevAdInx,tf.constant(self.wAds.shape[0])),axis=0)
        )
        
        
        t_rho = t_dotProds + t_scores - ( t_prevAdMask * tf.constant(1e5))
        
        t_maxInx = tf.argmax(t_rho)
        
        self.t_prevAdInx = t_prevAdInx
        self.t_probs = t_probs
        self.t_wAds = t_wAds
        self.t_w = t_w
        
        self.t_maxInx = t_maxInx
        
        
    def getW(self):
        return (self.c + self.a)/(self.v + self.a + self.b)
    
    def resetW(self):
        self.c[:] = 0
        self.v[:] = 0   
    
    def subSetIteration(self,probs,prevAdInx):
        w = self.getW()
        
        prevAdSum = 1+self.wAds[prevAdInx].sum(axis=0)
        prevProbSum = probs[prevAdInx].sum()
       
        newAs = np.log(self.wAds + prevAdSum)  
    
        dotProds = (w * newAs).sum(axis=1)
        
        scores = prevProbSum + probs

        rho = dotProds + scores
        rho[prevAdInx] = -np.inf
        
        maxInx = np.argmax(rho)
        
        return maxInx
        
        
    def getSubSet(self,userInx,n=6):
#         t = time.time()
        probs = self.regrModel.predict([
            np.array([userInx]*self.wAds.shape[0]),
            np.arange(self.wAds.shape[0])
        ],batch_size=50000).ravel()
#         print(time.time()-t)
        
        currAdSet = np.empty(0,dtype=np.int)
#         currAdSet = np.array([1])
        
        with tf.device('/gpu:0'):
            with tf.Session() as sess:

                while len(currAdSet) < n:
#                     t = time.time()
                    newAd = sess.run(self.t_maxInx,feed_dict={
                        self.t_prevAdInx: currAdSet,
                        self.t_probs: probs,
                        self.t_wAds: self.wAds,
                        self.t_w: self.getW()
                    })
#                     print(time.time()-t)
#                     print(newAd)
#                     newAd = self.subSetIteration(probs,currAdSet)
#                     print(list(newAd)[:50])
#                     break 
                    currAdSet = np.append(currAdSet,newAd)
        
        # Update v
        self.v += self.wAds[currAdSet].sum(axis=0)
            
        return currAdSet
    
    def registerClick(self,adInx):
        self.c += self.wAds[adInx]
        
smd = SubModDiv(sparseAdWeights,model)

import time

t = time.time()
print(smd.getSubSet(1))
print(time.time()-t)
    

[  2149   6531 135434  46596  16628   1172]
2.7590551376342773


In [25]:



class SubModDivUser():
    def __init__(self, nUsers, wAds, regrModel, alpha=None):
        self.regrModel = regrModel
        nCat = wAds.shape[1]
        if alpha == None:
            self.a = np.ones(nCat)
        else:
            assert(len(alpha) == nCat)
            self.a = alpha
        self.wAds = wAds.toarray()
        self.c = np.zeros((nUsers, wAds.shape[1])) # "c" in section 4.1 of paper
        
#         with tf.device(' ')
        with tf.device('/gpu:0'):
            self.initTensorflowOp()
        
    def initTensorflowOp(self):
        ## Tensorflow code for subsetiteration
        t_prevAdInx = tf.placeholder(tf.int32,shape=(None))
        t_probs = tf.placeholder(tf.float32,shape=(self.wAds.shape[0]))
        t_wAds = tf.placeholder(tf.float32,shape=self.wAds.shape)
        t_w = tf.placeholder(tf.float32,shape=(self.wAds.shape[1]))
        
        t_prevAdSum = tf.constant(1.0)+tf.reduce_sum(tf.gather(t_wAds,t_prevAdInx),axis=0)
        
        t_prevProbSum = tf.reduce_sum(tf.gather(t_probs,t_prevAdInx),axis=0)
        
        t_newAs = tf.log(t_wAds + t_prevAdSum)
        
        t_dotProds = tf.reduce_sum(t_w * t_newAs,axis=1)
        
        t_scores = t_prevProbSum + t_probs
        
        t_prevAdMask = tf.cond(
            tf.equal( tf.shape(t_prevAdInx)[0], tf.constant(0,dtype=tf.int32) ),
            lambda: tf.zeros(self.wAds.shape[0]),
            lambda: tf.reduce_sum(tf.one_hot(t_prevAdInx,tf.constant(self.wAds.shape[0])),axis=0)
        )
        
        
        t_rho = t_dotProds + t_scores - ( t_prevAdMask * tf.constant(1e5))
        
        t_maxInx = tf.argmax(t_rho)
        
        self.t_prevAdInx = t_prevAdInx
        self.t_probs = t_probs
        self.t_wAds = t_wAds
        self.t_w = t_w
        
        self.t_maxInx = t_maxInx
        
        
    def getW(self, userInx):
        return (self.c[userInx] + self.a) / np.linalg.norm(self.c[userInx] + self.a, ord=1)
    
    def resetW(self):
        self.c[:,:] = 0   
    
    def subSetIteration(self,probs,prevAdInx):
        w = self.getW()
        
        prevAdSum = 1+self.wAds[prevAdInx].sum(axis=0)
        prevProbSum = probs[prevAdInx].sum()
       
        newAs = np.log(self.wAds + prevAdSum)  
    
        dotProds = (w * newAs).sum(axis=1)
        
        scores = prevProbSum + probs

        rho = dotProds + scores
        rho[prevAdInx] = -np.inf
        
        maxInx = np.argmax(rho)
        
        return maxInx
        
        
    def getSubSet(self, userInx, n=6):
#         t = time.time()
        probs = self.regrModel.predict([
            np.array([userInx]*self.wAds.shape[0]),
            np.arange(self.wAds.shape[0])
        ],batch_size=50000).ravel()
#         print(time.time()-t)
        
        currAdSet = np.empty(0,dtype=np.int)

        
        with tf.device('/gpu:0'):
            with tf.Session() as sess:

                while len(currAdSet) < n:
#                     t = time.time()
                    newAd = sess.run(self.t_maxInx,feed_dict={
                        self.t_prevAdInx: currAdSet,
                        self.t_probs: probs,
                        self.t_wAds: self.wAds,
                        self.t_w: self.getW(userInx)
                    })

                    currAdSet = np.append(currAdSet,newAd)

        
#         print(self.wAds[currAdSet])
            
        return currAdSet
    
    def registerClick(self, userInx, adInx):
        self.c[userInx] += self.wAds[adInx]
        
smd = SubModDivUser(uniqUser.shape[0], sparseAdWeights, model)

t = time.time()
print(smd.getSubSet(1))
print(time.time()-t)
    

[ 2149  6531 17206 25614   158 22582]
2.833284854888916


In [6]:
class ItemSetDist():
    
    def __init__(self, wAds, alpha=1.0, beta=1.0):
        
        self.a = alpha
        self.b = beta
        self.wAds = wAds.toarray() # "a_i" in paper
        self.c = np.zeros(wAds.shape[1]) # "c" in section 4.1 of paper
        self.v = np.zeros(wAds.shape[1]) # "v" in section 4.1 of paper
        
    def getMostRel(self):
        
        w = self.getW()
        maxInx = np.argmax(np.dot(self.wAds, w))
        return maxInx
        
    def getW(self):
        
        return (self.c + self.a) + (self.v + self.a + self.b)
    
    def subsetIteration(self, prevadInx):
        
        S = self.wAds[prevadInx]
        X = self.wAds#[np.isin(np.arange(len(self.wAds)), currAd, invert=True)]
        
        maxInx = np.argmax(np.min(np.linalg.norm((S - np.repeat(X.reshape(-1, 1, 96), len(prevadInx), axis=1)), axis=-1), axis=-1))
        return maxInx
    
    def getSubset(self, n=6):
        
        currAdSet = []
        
        currAdSet.append(self.getMostRel())
        
        for i in range(n-1):
            
            maxInx = self.subsetIteration(currAdSet)
            currAdSet.append(maxInx)
            
                # Update v
            self.v += self.wAds[currAdSet].sum(axis=0)
        return currAdSet

In [7]:
isd = ItemSetDist(sparseAdWeights)

In [11]:
import time
t = time.time()
isd.getSubset()
print(time.time() - t)

12.12611198425293


In [27]:
currAd = [1,2,3]

In [29]:
wAds = sparseAdWeights.toarray()

In [37]:
wAds.shape

(559583, 96)

In [38]:
S = wAds[currAd]
X = wAds[np.isin(np.arange(len(wAds)), currAd, invert=True)]

In [53]:
np.linalg.norm((S - np.repeat(X.reshape(-1, 1, 96), 3, axis=1)), axis=-1).shape

(559580, 3)

In [55]:
np.argmax(np.min(np.linalg.norm((S - np.repeat(X.reshape(-1, 1, 96), 3, axis=1)), axis=-1), axis=-1))

2

In [25]:
np.dot(sparseAdWeights.toarray(), np.ones(sparseAdWeights.toarray().shape[1]))

(559583,)